# Anomaly Detection: Training

At the last step, we are doing anomaly detection. In this document, firstly the preprocessed masks and their marked anomalies (marked as 0 or 1, referred to as **text**) are used to train a sequential 3D CNN model. Later, predicted masks for the **Optovue** data loaded and their anomalies are predicted.

Import necessary libraries and mount the drive for data access.



In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv3D, Dense, Dropout, GlobalAveragePooling3D, MaxPooling3D
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load the Data

We have already saved the split data containing 100 sample of masks and binary values for anomalies named as **text**. Masks contain 20 slices, with each slice containing 6 layers of $512\times512$ masks.

In [ ]:
project_loc = '/content/drive/MyDrive/OCTA500/Project/Anomaly/'

In [ ]:
data_loc = project_loc + 'data/'

# Load the training data
train_data = np.load(data_loc + 'mt_train.npz')
X_train = train_data['mask']  # Masks
y_train = train_data['text']  # Texts

# Load the testing data
test_data = np.load(data_loc + 'mt_test.npz')
X_test = test_data['mask']  # Masks
y_test = test_data['text']  # Texts

# Slice amount
slices = X_test.shape[1]

In [ ]:
# Reshape the layers for CNN
X_train = X_train.reshape(-1, slices, 512, 512, 6)
X_test = X_test.reshape(-1, slices, 512, 512, 6)

## Build the Model

The model we are using is sequential 3D CNN. Our optimizer is Adam.

In [ ]:
lr = 0.01
momentum_value = 0.95

# Use SGD with Momentum optimizer
optimizer = SGD(learning_rate=lr, momentum=momentum_value)

loss = 'binary_crossentropy'
metrics = ['accuracy']

In [ ]:
model = Sequential([
    Conv3D(8, kernel_size=(3, 3, 3), activation='relu', input_shape=(50, 512, 512, 6), strides=(2, 2, 2)),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2, 2, 2)),
    Dropout(0.5),
    GlobalAveragePooling3D(),
    Dense(16, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

### Use a Generator

Since the dimensions are too large, even the **A100** GPU of Google Colab with **40 GB memory** crashes because of the limited resources. Using a generator is one of the ways to fix this issue, without downsampling.

Define a data generator.

In [ ]:
def data_generator(X_data, y_data, batch_size):
    num_samples = X_data.shape[0]
    while True:  # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            # Get the data for this batch
            batch_X = X_data[offset:offset + batch_size]
            batch_y = y_data[offset:offset + batch_size]

            yield batch_X, batch_y

Split training and validation data.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [ ]:
batch_size = 4

train_generator = data_generator(X_train, y_train, batch_size)
validation_generator = data_generator(X_val, y_val, batch_size)

### Add a Learning Rate Scheduler

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * np.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)

### Fit the model

In [ ]:
epochs = 200
patience = 25

steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_val) // batch_size

In [ ]:
# ModelCheckpoint callback
model_checkpoint = ModelCheckpoint(
    project_loc + 'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

In [ ]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

# Compute class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: weight for i, weight in enumerate(class_weights)}

In [ ]:
# Fit the model
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=epochs,
    class_weight=class_weights,
    callbacks=[early_stopping, lr_scheduler, model_checkpoint]
    )

Epoch 1/200
50/50 [==============================] - ETA: 0s - loss: 0.7001 - accuracy: 0.4950
Epoch 1: val_accuracy improved from -inf to 0.70000, saving model to /content/drive/MyDrive/OCTA500/Project/Anomaly/best_model.h5
50/50 [==============================] - 12s 144ms/step - loss: 0.7001 - accuracy: 0.4950 - val_loss: 0.6762 - val_accuracy: 0.7000 - lr: 0.0100
Epoch 2/200


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


50/50 [==============================] - ETA: 0s - loss: 0.6832 - accuracy: 0.6212
Epoch 2: val_accuracy did not improve from 0.70000
50/50 [==============================] - 7s 127ms/step - loss: 0.6832 - accuracy: 0.6212 - val_loss: 0.6892 - val_accuracy: 0.5789 - lr: 0.0100
Epoch 3/200
50/50 [==============================] - ETA: 0s - loss: 0.6813 - accuracy: 0.6212
Epoch 3: val_accuracy did not improve from 0.70000
50/50 [==============================] - 6s 119ms/step - loss: 0.6813 - accuracy: 0.6212 - val_loss: 0.6925 - val_accuracy: 0.5789 - lr: 0.0100
Epoch 4/200
50/50 [==============================] - ETA: 0s - loss: 0.7005 - accuracy: 0.5556
Epoch 4: val_accuracy did not improve from 0.70000
50/50 [==============================] - 6s 119ms/step - loss: 0.7005 - accuracy: 0.5556 - val_loss: 0.6908 - val_accuracy: 0.5789 - lr: 0.0100
Epoch 5/200
50/50 [==============================] - ETA: 0s - loss: 0.6513 - accuracy: 0.6970
Epoch 5: val_accuracy did not improve from 0.70